<img src="assets/socalo-ICDA.png">

# Python para Finanzas y Ciencia de Datos
Federico Brun | fedejbrun@gmail.com

_Jueves 22 Octubre 2020_

## EDA con Python

<img src="https://files.realpython.com/media/Descriptive-Statistics-in-Python_Watermarked.fec81e9a41f9.jpg">

_Fuente: realpython.com_

El **Análisis Exploratorio de Datos** (EDA) es una forma de analizar datos, y consiste en _el tratamiento estadísitco al que se someten las muestras recogidas durante un proceso de investigación en cualquier campo científico_  , o de la Ciencia de Datos.

Como ya hemos discutido, al usar Python contamos con un set de herramientas especializadas en las actividades que implica dicho tratamiento de los datos:
* Pandas y Numpy para la obtención, organización, procesamiento y limpieza del conjunto de datos a analizar.
* Matplotlib, Seaborn y Plotly parta la visualización del conjunto de dicho conjunto de datos.

## EDA sobre Dataset de Kaggle

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation

La competición consiste en construir un modelo de Machine Learning (ML) para predecir el precio de diferentes inmuebles.

Lo que haremos sera el **paso previo**, que consiste en llevar a cabo un análisis EDA sobre los datos, que posteriormente podrían ser utilizados para generar un modelo predictivo de ML.

**EL OBJETIVO** de este ejercicio , NO es llegar a alguna conclusión, sino **ANALIZAR** los datos utilizando las diferentes **HERRAMIENTAS** presentadas en el curso.

## 1. Exploración de datos
#### 1.1 Visualización de datos sin procesar

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('clase8_train.csv')
df.head()

Más informacion de nuestro conjunto de datos

In [ ]:
df.info()

A partir de esta información, ya podemos ver que hay algunos datos que no van a ser relavantes en nuestro análisis exploratorio, dado que existen muchos datos incompletos (por ejemplo `Alley` y `PoolQC`).

#### 1.2 Eliminar valores nulos

Como el DataFrame ya nos provee un índice, vamos a limpiar el DataFrame eliminando la columna `Id` y todas las columnas o _feautures_ de los cuales exista una cantidad de datos vacios del 30%  mayor. 

Para conocer qué dato representa cada categoría (columnas), lo buscamos en el archivo `clase8_data_description.txt`.

In [ ]:
# La funcion df.count() no contbiliza valores vacios
df2 = df[[column for column in df if df[column].count() / len(df) >= 0.3]]
del df2['Id']
print("Lista de columnas eliminadas:", end=" ")
for c in df.columns:
    if c not in df2.columns:
        print(c, end=", ")
print('\n')
df = df2

Para armar la lista de columnas a eliminar, en la línea 2 usamos una sintaxis particular denominada <a href="https://j2logo.com/list-comprehensions-en-python/" target="_blank">Comprensión de lista</a>, que nos permite definir una lista usando un bucle `for`, en una sola línea.

In [ ]:
df.info()

A continuación veamos cómo se distribuye el precio de venta en la muestra completa.

In [ ]:
print(df['SalePrice'].describe())

In [ ]:
print(df['SalePrice'].describe())

plt.style.use('bmh')

plt.figure(figsize=(15, 8))
sns.distplot(df['SalePrice'], color='g', bins=100);

Vemos que existe una asimetría con mayor densidad hacia la izquierda, y encontramos algunos valores extremos por encima de los USD 500,000 que eventualmente podríamos quitar del conjunto de datos, para obtener una distribución normal.

#### Distribucón numérica

Ahora vamos a evaluar cómo se distribuyen numéricamente, todas las características del conjunto de datos, para poder visualizarlas mas claramente.

Primero veamos los diferentes tipos de datos que guardamos en el DataFrame.

In [ ]:
list(set(df.dtypes.tolist()))

_La función `list()` convierte el paráemtro que recibe en una lista._

_La función `set()` convierte el parámetro que recibe (un objeto iterable) en un set <a href="https://www.geeksforgeeks.org/python-set-method/#:~:text=set()%20method%20is%20used,dintinct%20elements%2C%20commonly%20called%20Set.&text=Parameters%20%3A%20Any%20iterable%20sequence%20like,if%20no%20element%20is%20passed." target="_blank">Tutorial set()</a> ._

Vamos a usar la lista anterior para filtrar solo los datos numéricos del DataFrame.

In [ ]:
df_num = df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

Y ahora visualizamos un histograma para cada característica disponible.

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8); 

Observamos que características como `1stFlrSF`, `TotalBsmtSF`, `LotFrontage`, `GrLiveArea` parecen compartir distribuciones similares a la distribución que muestra `SalePrice`; algo para tener en cuenta para mas adelante.

Recordemos el objetivo de este análisis EDA: Preparar los datos para entrenar un modelo de ML para **predecir el precio de un inmueble**. Con esto en mente, cobra mayor relevancia la observación del párrafo anterior.

#### 1.3 Buscar correlaciones

A continuación, veamos qué caracterísitcas se encuentran más fuertemente correlacionadas con `SalePrice` y en qué medida.

Recordemos que la correlación entre dos variables nos dá un indicio de qué tan relacionadas están dichas variables.

In [ ]:
df_num_corr = df_num.corr()['SalePrice'][:-1] # -1 porque la ultima fila es SalePrice

golden_features_list = df_num_corr[abs(df_num_corr) > 0.5].sort_values(ascending=False)
print("Existen {} características fuertemente correlacionadas con SalePrice:\n{}".format(len(golden_features_list), golden_features_list))

Ahora contamos con una lista de características correlacionadas con `SalePrice`.

Sin embargo, como veremos mas adelante en el curso, sabemos que los valores extremos de un conjunto de datos, pueden llevarnos a una conclusión incorrecta.

Seguidamente vamos a eliminar estos valores extremos del conjunto de datos y repetir el proceso de buscar correlaciones entre las diferentes características y `SalePrice`. _(No vamos a profundizar en esta explicación ahora)._  


In [ ]:
import operator

individual_features_df = []
for i in range(0, len(df_num.columns) - 1): # -1 because the last column is SalePrice
    tmpDf = df_num[[df_num.columns[i], 'SalePrice']]
    tmpDf = tmpDf[tmpDf[df_num.columns[i]] != 0]
    individual_features_df.append(tmpDf)

all_correlations = {feature.columns[0]: feature.corr()['SalePrice'][0] for feature in individual_features_df}
all_correlations = sorted(all_correlations.items(), key=operator.itemgetter(1))
for (key, value) in all_correlations:
    print("{:>15}: {:>15}".format(key, value))

In [ ]:
from pprint import pprint

golden_features_list =  [key for key, value in all_correlations if abs(value) >= 0.5]
print("Existen {} características fuertemente correlacionadas con SalePrice:\n".format(len(golden_features_list)))
pprint(golden_features_list)

In [ ]:
all_correlations_df = pd.DataFrame().from_dict(all_correlations)
all_correlations_df.set_index(0, inplace=True)

# pprint(all_correlations_df)

golden_features_list = all_correlations_df[abs(all_correlations_df[1]) > 0.5].sort_values(by=1, ascending=False)

print("Nivel de correlación con SalePrice:\n{}".format(golden_features_list))

Al ver la correlación entre las distribuciones numéricas de las diferentes características y `SalesPrice` encontramos que existe 11 caracterísitcas que presentan una relación fuerte con el precio de venta del inmueble.

A continuación vamos a Visualizar la correlación entre todas las características, y aplicando una máscara vamos a ver aquellas cuyo coeficiente de correlación sea mayor a 0.5 y menor a -0.4 ; para ver si encontramos algun otro tipo de relación aparte de el precio de venta del inmueble.

In [ ]:
corr = df_num.drop('SalePrice', axis=1).corr() # Ya examinamos correlacion con SalesPrice

plt.figure(figsize=(16, 14))

sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.4)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

Vemos por ejemplo que hay una fuerte correlación postivia entre la superficie de la planta baja del inmueble y la superficie del sótano (`1stFlrSF/TotalBsmtSF`).

In [ ]:
corr.loc['1stFlrSF', 'TotalBsmtSF']

### 2. Análisis cuantitativo
Veamos ahora las propiedades cuantitativas de nuestro conjunto de datos, para aplicar más herramientas de las que ya conocemos.

Para empezar, vemos que algunas de las características descritas en el conjunto de datos son categóricas, y otras son cuantitativas.

Vamos a separar unas de otras valiéndonos de la información provista en `clase8_data_description.text`.

Construimos una lista de todas las variables cuantitativas del conjunto de datos.

In [ ]:
quantitative_features_list = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice']
df_quantitative_values = df[quantitative_features_list]
df_quantitative_values.head()

A partir de esta lista que representa un subconjunto de datos a analizar, veamos aquellas que estan fuertemente correlacionadas a `SalePrice`.

In [ ]:
golden_features_list = [key for key, value in all_correlations if abs(value) >= 0.5]

features_to_analyse = [x for x in quantitative_features_list if x in golden_features_list]
features_to_analyse.append('SalePrice')
features_to_analyse

Ahora veamos las distribuciones de cada variable, junto a una regresion lineal ajustada para cada una.
Recordemos el objetivo de este ejercicio.

In [ ]:
fig, ax = plt.subplots(round(len(features_to_analyse) / 3), 3, figsize = (18, 12))

for i, ax in enumerate(fig.axes):
    if i < len(features_to_analyse) - 1:
        sns.regplot(x=features_to_analyse[i],y='SalePrice', data=df[features_to_analyse], ax=ax)

Podemos observar que características como `TotalBsmtSF`, `1stFlrSF`, `GrLivArea` presentan grandes dispersiones.

Veamos ahora por ejemplo, que porcentaje de todos los inmuebles del conjunto de datos, cuentan con conchera para 1, 2, 3, 4 o nigún auto:

In [ ]:
two_cars_garage = df_quantitative_values.groupby(['GarageCars'], as_index=False).count()
two_cars_garage = two_cars_garage['SalePrice']

In [ ]:
import plotly.express as px 

fig = px.pie(two_cars_garage, values=['SalePrice'], names=['SalePrice']) 
fig.show()
